In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline


import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
from tensorboardX import SummaryWriter
torch.cuda.set_device(1)


In [2]:
# Add previous directory
import sys 
sys.path.append('..')

# import loader
from loader import NiftiDataset
import transforms as tfs

In [3]:
parser = argparse.ArgumentParser(description='PyTorch MNIST Example')
parser.add_argument('--batch-size', type=int, default=64, metavar='N',
                    help='input batch size for training (default: 64)')
parser.add_argument('--test-batch-size', type=int, default=1000, metavar='N',
                    help='input batch size for testing (default: 1000)')
parser.add_argument('--epochs', type=int, default=2, metavar='N',
                    help='number of epochs to train (default: 10)')
parser.add_argument('--lr', type=float, default=0.01, metavar='LR',
                    help='learning rate (default: 0.01)')
parser.add_argument('--momentum', type=float, default=0.5, metavar='M',
                    help='SGD momentum (default: 0.5)')
parser.add_argument('--no-cuda', action='store_true', default=False,
                    help='disables CUDA training')
parser.add_argument('--seed', type=int, default=1, metavar='S',
                    help='random seed (default: 1)')
parser.add_argument('--log-interval', type=int, default=10, metavar='N',
                    help='how many batches to wait before logging training status')
# if __name__=='__main__':
#     args = parser.parse_args()
# else:
args = parser.parse_args("--batch-size 1".split())
args.cuda = not args.no_cuda and torch.cuda.is_available()

In [4]:
kwargs = {'num_workers': 1, 'pin_memory': True}
train_loader = torch.utils.data.DataLoader(NiftiDataset(csv_file='/media/df3-dafna/Netanell/BLLAMODEL/Data.xlsx',
                                    root_dir='/media/df3-dafna/Netanell/BLLAMODEL/',
                                           transform=transforms.Compose([
                                               tfs.ToTensor(),
                                               tfs.Rescale((128,128))
                                           ])),
                                    batch_size=args.batch_size, shuffle=True, **kwargs)




In [5]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x)
    

In [6]:
model = Net()
if args.cuda:
    model.cuda()
optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=args.momentum)


def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        if args.cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data.item()))
            niter = epoch*len(train_loader)+batch_idx
            writer.add_scalar('Train/Loss', loss.data.item(), niter)


In [7]:
for epoch in range(1, args.epochs + 1):
    train(epoch)
    torch.save(model, '/tmp/model{}'.format(epoch))

132 140 128 128 torch.Size([132, 140, 24]) (128, 128, 24)
136 132 128 128 torch.Size([136, 132, 26]) (128, 128, 26)


AttributeError: 'str' object has no attribute 'cuda'

In [12]:
for batch_idx, (data, target) in enumerate(train_loader):
      print(target)

120 124 128 128 torch.Size([120, 124, 20]) (128, 128, 20)
TCD_Selection
160 132 128 128 torch.Size([160, 132, 28]) (128, 128, 28)
TCD_Selection
132 140 128 128 torch.Size([132, 140, 32]) (128, 128, 32)
TCD_Selection
136 124 128 128 torch.Size([136, 124, 34]) (128, 128, 34)
140 127 128 128 torch.Size([140, 127, 20]) (128, 128, 20)
TCD_Selection
TCD_Selection
136 132 128 128 torch.Size([136, 132, 26]) (128, 128, 26)
TCD_Selection
136 124 128 128 torch.Size([136, 124, 34]) (128, 128, 34)
108 120 128 128 torch.Size([108, 120, 18]) (128, 128, 18)
TCD_Selection
TCD_Selection
120 112 128 128 torch.Size([120, 112, 27]) (128, 128, 27)
124 140 128 128 torch.Size([124, 140, 22]) (128, 128, 22)
TCD_Selection
TCD_Selection
124 136 128 128 torch.Size([124, 136, 22]) (128, 128, 22)
132 140 128 128 torch.Size([132, 140, 24]) (128, 128, 24)
TCD_Selection
TCD_Selection
116 104 128 128 torch.Size([116, 104, 22]) (128, 128, 22)
128 116 128 128 torch.Size([128, 116, 20]) (128, 128, 20)
TCD_Selection
TCD_Se